##基于高德地图API的路径规划

查看高德API key: https://console.amap.com/dev/key/app

以南京地铁为例

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
city = '南京'                       ###### 指定城市名
csv_file = './subways_Nanjing.csv'            ###### 保存的csv文件名

# Step 1: 数据采集
def get_page_content(url):
  html = requests.get(url, headers=header, timeout=10)
  content = html.text
  soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
  return soup

url = 'https://ditie.mapbar.com/nanjing_line/'       ####### 指定某一城市的地铁线路网址
soup = get_page_content(url)

# name, site
df = pd.DataFrame(columns=['name', 'site'])
subways = soup.find_all('div', class_='station')
for subway in subways:
  route = subway.find('strong', class_='bolder').text  # 每次遍历得到一条线路
  stations = subway.find('ul').find_all('a')  # 每条线路的ul里面有多个a，代表各个站点
  for station in stations:
    temp = {'name': station.text, 'site': route}
    #print(temp)
    df = df.append(temp, ignore_index=True)

df['city'] = city
df.to_csv(csv_file, index=False, encoding='utf-8')
df.head()

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,name,site,city
0,迈皋桥站,地铁1号线,南京
1,红山动物园站,地铁1号线,南京
2,南京站,地铁1号线,南京
3,新模范马路站,地铁1号线,南京
4,玄武门站,地铁1号线,南京


##Step 2： 获取地点的坐标（经纬度）

使用高德地图API，根据用户申请的key，获取指定地点的所在经度、维度，JSON格式是字典

https://restapi.amap.com/v3/place/text?keywords=五道口&city=北京&output=json&offset=20&page=1&key=e10ee1784909b7d58db5d008b30d8989&extensions=all 


In [ ]:
# Step 2： 获取地点的坐标（经纬度）
# 使用高德地图API，根据用户申请的key，获取指定地点的所在经度、维度，JSON格式是字典
# https://restapi.amap.com/v3/place/text?keywords=五道口&city=北京&output=json&offset=20&page=1&key=e10ee1784909b7d58db5d008b30d8989&extensions=all 

import re
# 通过keyword, city得到location
def get_location(keyword, city):
  url = 'https://restapi.amap.com/v3/place/text?keywords='+keyword+'&city='+city+'&output=json&offset=20&page=1&key=e10ee1784909b7d58db5d008b30d8989&extensions=all'
  print(url)
  data = requests.get(url, headers=header, timeout=20)
  data.encoding = 'utf-8'
  data = data.text
  
  # "location":"116.73891,40.08394"
  # .*具有贪婪模式，匹配到不能匹配为止
  # 加一个?表示懒惰模式，经过一个匹配后，当前的结束
  pattern = '"location":"(.*?),(.*?)"'
  result = re.findall(pattern, data)
  print(result[0][0], result[0][1])
  try:
    return result[0][0], result[0][1]
  except:
    get_location(keyword.replace('站', ''), city)

#get_location('五道口', '北京')
df = pd.read_csv(csv_file, index_col=None)
df['longitude'], df['latitude'] = None, None  # 表格中添加经纬度列

In [ ]:
for index, row in df.iterrows():
  if row['longitude'] == None:
    name, city = str(row['name']), str(row['city'])
    #print(name, city)
    try:
      longi, lati = get_location(name, city)
      df.iloc[index]['longitude'] = longi
      df.iloc[index]['latitude'] = lati
    except:
      continue
  else:
    continue
df.to_csv(csv_file, index=False)

##Step 3: 计算两点之间的距离

使用高德地图API，获取指定两点之间的距离，比如从五道口到知春路

http://restapi.amap.com/v3/distance?key=e10ee1784909b7d58db5d008b30d8989&origins=116.337581,39.993138&destination=116.339941,39.976228&type=1

五道口的坐标(116.337581,39.993138)，
知春路的坐标(116.339941,39.976228)


In [ ]:
import pandas as pd
import re, requests

header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}

def compute_distance(long1, lati1, long2, lati2):
  url = 'http://restapi.amap.com/v3/distance?key=e10ee1784909b7d58db5d008b30d8989&origins='+str(long1)+\
    ','+str(lati1)+'&destination='+str(long2)+','+str(lati2)+'&type=1'
  print(url)
  data = requests.get(url, headers=header, timeout=10)
  data.encoding = 'utf-8'
  data = data.text
  pattern = "\"distance\":\"(.*?)\",\"duration\":\"(.*?)\""  # 两点间距离和时间
  result = re.findall(pattern, data)
  #print(result[0][0])
  return result[0][1]

# 数据加载
data = pd.read_csv('./subways_Nanjing.csv', index_col=None)
#data.head() 

from collections import defaultdict
# 创建图中两点间的最短距离
graph = defaultdict(dict)
for i in range(data.shape[0]):
  site1 = data.iloc[i]['site']
  if i<(data.shape[0]-1):
    site2 = data.iloc[i+1]['site']
    # 如果是同一条线路
    if site1 == site2:  
      name1, name2 = data.iloc[i]['name'], data.iloc[i+1]['name']
      graph[name1][name2] = None
      graph[name2][name1] = None

In [ ]:
for i in range(data.shape[0]):
  site1 = data.iloc[i]['site']
  if i<data.shape[0]-1:
    site2 = data.iloc[i+1]['site']
    # 如果是同一条线路
    if site1 == site2:
      long1, lati1 = data.iloc[i]['longitude'], data.iloc[i]['latitude']
      long2, lati2 = data.iloc[i+1]['longitude'], data.iloc[i+1]['latitude']
      name1, name2 = data.iloc[i]['name'], data.iloc[i+1]['name']
      if graph[name1][name2] == None or graph[name2][name1] == None:
        try:
          distance = compute_distance(long1, lati1, long2, lati2)
          graph[name1][name2] = distance
          graph[name2][name1] = distance
          print(name1, name2, distance)
        except:
          continue
# 保存python对象
import pickle
output = open('graph.pkl', 'wb')
pickle.dump(graph, output)
print(graph)

defaultdict(<class 'dict'>, {'迈皋桥站': {'红山动物园站': '420'}, '红山动物园站': {'迈皋桥站': '420', '南京站': '600'}, '南京站': {'红山动物园站': '600', '新模范马路站': '480', '小市站': '780', '南京林业大学新庄站': '480'}, '新模范马路站': {'南京站': '480', '玄武门站': '300'}, '玄武门站': {'新模范马路站': '300', '鼓楼站': '420'}, '鼓楼站': {'玄武门站': '420', '珠江路站': '600', '鸡鸣寺站': '420', '云南路站': '180'}, '珠江路站': {'鼓楼站': '600', '新街口站': '420'}, '新街口站': {'珠江路站': '420', '张府园站': '540', '上海路站': '240', '大行宫站': '660'}, '张府园站': {'新街口站': '540', '三山街站': '360'}, '三山街站': {'张府园站': '360', '中华门站': '660'}, '中华门站': {'三山街站': '660', '安德门站': '600'}, '安德门站': {'中华门站': '600', '天隆寺站': '120', '小行站': '540'}, '天隆寺站': {'安德门站': '120', '软件大道站': '300'}, '软件大道站': {'天隆寺站': '300', '花神庙站': '240'}, '花神庙站': {'软件大道站': '240', '南京南站': '720'}, '南京南站': {'花神庙站': '720', '双龙大道站': '360', '翠屏山站': '840', '明发广场站': '660', '宏运大道站': '600'}, '双龙大道站': {'南京南站': '360', '河定桥站': '240'}, '河定桥站': {'双龙大道站': '240', '胜太路站': '420'}, '胜太路站': {'河定桥站': '420', '百家湖站': '180'}, '百家湖站': {'胜太路站': '180', '小龙湾站': '600'}, '小龙湾站': {'百家湖站': '6

##Step 4: 路径规划
使用Dijkstra计算最优路径

In [ ]:
# 直接调用pkl
import pickle
file = open('graph.pkl', 'rb')
graph = pickle.load(file)

# 找到开销最小的节点
def find_lowest_cost_node(costs):
  # 初始化数据
  lowest_cost = float('inf')
  lowest_cost_node = None
  # 遍历所有节点
  for node in costs:
    # 如果节点没有被处理
    if node not in processed:
      if costs[node] < lowest_cost:  # 当前值更小，更新lowest
        lowest_cost = costs[node]
        lowest_cost_node = node
  return lowest_cost_node

# 找到最短路径，从终点开始
def find_shortest_path():
  node = end
  shortest_path = [end]
  while parents[node] != start:
    shortest_path.append(parents[node])
    node = parents[node]
  shortest_path.append(start)
  return shortest_path

# 计算图中从start到end的最短路径
def dijkstra():
  # find the lowest cost node
  node = find_lowest_cost_node(costs)
  #print('当前cost最小的节点：', node)
  while node is not None:
    # 获取节点目前的cost
    cost = costs[node]
    # 获取节点的邻居
    neighbors = graph[node]
    # 遍历所有的邻居，看是否更新cost
    for neighbor in neighbors.keys():
      new_cost = cost + float(neighbors[neighbor])
      if neighbor not in costs or new_cost < costs[neighbor]:
         costs[neighbor] = new_cost    # 更新cost
         parents[neighbor] = node    # 更新parent
    # 标记当前节点为已处理
    processed.append(node)
    # 重复循环
    node = find_lowest_cost_node(costs)
  # 所有节点处理完毕，找最短距离
  shortest_path = find_shortest_path()
  shortest_path.reverse() 
  print('从{}到{}的最短路径：\n{}'.format(start, end, shortest_path))
  print('所需时间为：{}分钟'.format(costs[end]/60))
  print('需经过{}站'.format(len(shortest_path)))

def compute(site1, site2):
  # 设置出发点和终点
  global start, end, costs, parents, processed
  start, end = site1, site2

  # 创建costs字典存储所有节点的cost，cost指从起点到该节点的距离
  costs = {}
  costs[start] = 0    # 初始节点cost为0

  # 存储父节点的Hash表，用于记录路径
  parents = {} 

  # 记录处理过的节点list
  processed = []

  dijkstra() 

if __name__ == '__main__':
  site1, site2 = '南京站', '仙林中心站'
  #site1, site2 = '龙江站', '百家湖站'
  shortest_path = compute(site1, site2)

从南京站到仙林中心站的最短路径：
['南京站', '南京林业大学新庄站', '鸡鸣寺站', '九华山站', '岗子村站', '蒋王庙站', '王家湾站', '聚宝山站', '苏宁总部徐庄站', '金马路站', '仙鹤门站', '学则路站', '仙林中心站']
所需时间为：97.0分钟
需经过13站
